In [1]:
import pygame as pg
import sys
from pygame.locals import *
import numpy as np
from keras.models import load_model
import cv2 as cv

pygame 2.0.0.dev10 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
White=(255,255,255)
Black=(0,0,0)
Red=(255,0,0)
Boundary=20
WINDOWSIZEX=600
WINDOWSIZEY=500

IMAGESAVE=False

Model=load_model("./model.h5")

Labels={0:"ZERO",1:"ONE",2:"TWO",3:"THREE",4:"FOUR",5:"FIVE",6:"SIX",7:"SEVEN",8:"EIGHT",9:"NINE"}

In [3]:
#intialize pygame
pg.init()

FONT=pg.font.SysFont("arial",18)
DISPLAYSURFACE=pg.display.set_mode((WINDOWSIZEX,WINDOWSIZEY))
pg.display.set_caption("Digit Recognition")
iswriting=False

number_x=[]
number_y=[]
img_c=1
PREDICT=True

while True:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            pg.quit()
            sys.exit()
            
        if event.type == MOUSEMOTION and iswriting:
            x,y=event.pos
            pg.draw.circle(DISPLAYSURFACE, White, (x,y), 4, 0)
            number_x.append(x)
            number_y.append(y)
            
        if event.type == MOUSEBUTTONDOWN:
            iswriting=True
            
        if event.type == MOUSEBUTTONUP:
            iswriting=False
            number_x=sorted(number_x)
            number_y=sorted(number_y)
            
            rect_min_x,rect_max_x=max(number_x[0]-Boundary,0),min(WINDOWSIZEX,number_x[-1]+Boundary)
            rect_min_y,rect_max_y=max(number_y[0]-Boundary,0),min(WINDOWSIZEY,number_y[-1]+Boundary)
            
            number_x=[]
            number_y=[]
            
            img_arr=np.array(pg.PixelArray(DISPLAYSURFACE))[rect_min_x:rect_max_x,rect_min_y:rect_max_y].T.astype(np.float32)
            
            if IMAGESAVE:
                cv.imwrite("image.png")
                img_c+=1
            
            if PREDICT:
                image=cv.resize(img_arr,(28,28))
                image=np.pad(image,(10,10),'constant',constant_values=0)
                image=cv.resize(image,(28,28))/255
                
                label=str(Labels[np.argmax(Model.predict(image.reshape(1,28,28,1)))])
                
                textsurface=FONT.render(label,True,Red,White)
                textrec=textsurface.get_rect()
                textrec.left,textrec.bottom=rect_min_x,rect_max_y
                DISPLAYSURFACE.blit(textsurface,textrec)
                
                
            if event.type==KEYDOWN:
                if event.unicode=='n':
                    DISPLAYSURFACE.fill(Black)
                    
                    
    pg.display.update()

SystemExit: 

C:\Users\Sachin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
